# PVE-analysis on research funding
*Collective Compass Project, DJA, 2022*

---

This notebook contains a basic first data analysis of the data from the Collective Compass project. It shows the data categorized per institute, career stage, discipline, and gender. The variables `kt_*` contain the chosen allocation of research money per predefined category.

---
## Loading the necessary Python modules.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import requests
plt.rcParams.update({'font.size': 14}) # must set in top
plt.rcParams.update({'figure.figsize': (14, 4)}) # must set in top

---
## Download the data

In [ ]:
# download a fresh copy of the dataset from the OSF
data_url = 'https://osf.io/f76rb//?action=download'
response = requests.get(data_url)

if response.status_code == 200:
    with open('../data/dat.csv', 'wb') as f:
        f.write(response.content)

In [ ]:
df = pd.read_csv('../data/dat.csv')

---
## Plot basic participation stats

In [ ]:
plt.figure()
plt.subplot(121)
df['institution'].value_counts().plot.bar()
plt.grid()
plt.subplot(122)
df['versie'].value_counts().plot.bar();

In [ ]:
plt.figure()
df['institution'].value_counts().plot.bar()

In [ ]:
df_vals = df[['institution', 'sex']].value_counts()
df_vals.unstack('sex').plot.bar(stacked=True);

In [ ]:
plt.figure()
df['pos9'].value_counts().plot.bar()

In [ ]:
plt.figure()
df['discipline'].value_counts().plot.bar()

In [ ]:
df_vals = df[['pos9', 'sex']].value_counts()
df_vals.unstack('sex').plot.bar(stacked=True);

In [ ]:
df_vals = df[['discipline', 'sex']].value_counts()
df_vals.unstack('sex').plot.bar(stacked=True);

---
## Postprocess the kt values to fraction of the total budget.

In [ ]:
# Second, we divide the spending by the total funds available.
# to make each kt_funds* a fraction of the total money spend.
mask = df.columns.str.contains('funds_available|versie|pos5|kt_funds_*')
df_kt = df.iloc[:, mask].copy()
df_kt = df_kt.dropna()

for col in df_kt.iloc[:, df_kt.columns.str.contains('kt_funds*')]:
    df_kt.loc[:, col] /= df_kt.loc[:, 'funds_available']

---
## Show the relative funding allocation for several groups (total funding, career stage ...)

In [ ]:
# Create a table with the fraction spent per kt item per group.
kts = list( df_kt.iloc[:, df_kt.columns.str.contains('kt_funds*')] )
df_kt_per_version = df_kt.groupby('versie')[ kts ]
df_kt_per_version.mean()

In [ ]:
# Print the fraction of the total money spent per experiment version (with differennt total budgets).
df_kt_per_version.mean().sum(axis=1)

In [ ]:
# Plot a histogram with a bar per version.
df_kt_per_version.mean().transpose().plot.bar()
plt.grid()

In [ ]:
# Plot a histogram per version.
df_kt_per_version.mean().plot.bar(xlabel='total funding version');
plt.grid()
plt.legend(loc=0, ncol=3)

In [ ]:
# Create a table with the fraction spent per kt item per career stage.
kts = list( df_kt.iloc[:, df_kt.columns.str.contains('kt_funds*')] )
df_kt_per_pos5 = df_kt.groupby('pos5')[ kts ]
df_kt_per_pos5.mean()

In [ ]:
df_kt_per_pos5.mean().plot.bar(xlabel='career stage');
plt.grid()
plt.legend(loc=0, ncol=3)

In [ ]:
# Simplify the plot and sum categories
mask = df.columns.str.contains('pos5|kt_funds_*')
df_kt2 = df.iloc[:, mask].copy()
df_kt2 = df_kt2.dropna()

df_kt2['kt_competition'] = (
      df_kt2['kt_funds_first_ecr']
    + df_kt2['kt_funds_personal_ecr']
    + df_kt2['kt_funds_personal_senior']
    + df_kt2['kt_funds_small_first']
    + df_kt2['kt_funds_small_second']
    + df_kt2['kt_funds_team']
    + df_kt2['kt_funds_thematic'] )
df_kt2['kt_rolling'] = (
      df_kt2['kt_funds_rolling_ecr']
    + df_kt2['kt_funds_rolling_senior'] )

In [ ]:
kts = ['kt_competition', 'kt_rolling', 'kt_funds_award']
df_kt_per_pos5_summed = df_kt2.groupby('pos5')[ kts ]
df_kt_per_pos5_summed.mean()

In [ ]:
df_kt_per_pos5_summed.mean().plot.bar(xlabel='career stage');
plt.ylim(0, 400)
plt.grid()
plt.legend(loc=0, ncol=3);